In [1]:
# Todd McCullough
# 2020
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import cpl_main as cpl 
import random
import re

In [2]:
results = pd.read_csv(f'datasets/soccer-nn-train.csv')
results.head(2)

,game,team,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,r,s
0,I1,Forge FC,0.71,0.89,0.74,0.71,0.48,0.89,0.77,0.52,0.50,0.48,0.42,0.0,0.0,0.0,2,1
1,I1,York9 FC,0.93,0.92,0.91,0.86,0.41,0.78,0.65,0.64,0.46,0.70,0.47,0.0,0.0,0.0,2,1


In [3]:
gnb_db = results.copy()

In [4]:
a = []
for game in gnb_db['game'].unique():
    match = gnb_db[gnb_db['game'] == game]
    match = match.reset_index()
    for j in range(match.shape[0]):
        if j == 0:
            home = match.loc[j,['game','team','r','s']].values
            away = match.loc[j+1,['team','r','s']].values
            event = [home[0],home[1],home[2],home[3],away[0],away[1],away[2]]
            a.append(event)
        else:
            pass

In [5]:
print(results.shape,gnb_db.shape)

(3866, 18) (3866, 18)


In [6]:
db = pd.DataFrame(a,columns=['game','home','hr','hs','away','ar','as'])
db['hr'] = db['hr'].apply( lambda x: 'W' if x == 3 else 'D' if x == 2 else 'L')
db['ar'] = db['ar'].apply( lambda x: 'W' if x == 3 else 'D' if x == 2 else 'L')
db['d'] = db['game'].apply( lambda x: re.sub('[A-Z]+','',x))
db['m'] = 1

In [7]:
db.head(5)

,game,home,hr,hs,away,ar,as,d,m
0,I1,Forge FC,D,1,York9 FC,D,1,1,1
1,I2,Pacific FC,W,1,HFX Wanderers FC,L,0,2,1
2,I3,Pacific FC,L,1,Valour FC,W,2,3,1
3,I4,HFX Wanderers FC,W,2,Forge FC,L,1,4,1
4,I5,Cavalry FC,W,2,York9 FC,L,1,5,1


In [8]:
q1,q2 = 'Toronto FC','Columbus'
t1_x,t1_y = cpl.get_NB_data(db,q1)
t2_x,t2_y = cpl.get_NB_data(db,q2)
home_win, draw, away_win = cpl.get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)
print(f'{q1} home win probability: {round(home_win,3)} \n{q2} away win probability: {round(away_win,3)} \nDraw probability: {round(draw,3)}')

Toronto FC home win probability: 0.43 
Columbus away win probability: 0.42 
Draw probability: 0.35


In [9]:
if home_win > away_win:
    hp, ap = 3, 1
elif home_win < away_win:
    hp, ap = 1, 3
else:
    hp, ap = 2, 2

In [10]:
hp, ap

(3, 1)

In [11]:
def get_team_comparison(data,q1,q2):
    # getting games with q1 in both home or away
    db = data[(data['home'] == q1) | (data['away'] == q1)]
    db = db.reset_index()
    db.pop('index')
    # filering down more to get only the games against q2
    db = db.sort_values(by=['m','d'],ascending=False)
    db = db[(db['home'] == q2) | (db['away'] == q2)]
    #print(db)
    db = db.reset_index()
    db.pop('index')
    if db.empty == True:
        db = pd.DataFrame([(0,0,0,0,q1,'D',q2,'D','empty',q1)],columns=['d','m','hs','as','home','hr','away','ar','summary','team'])
    if db.shape[0] > 15:
        return db.head(15)
    else:
        return db

In [12]:
games = get_team_comparison(db.head(3800),'Forge FC','Cavalry FC')
games

,game,home,hr,hs,away,ar,as,d,m
0,I93,Forge FC,W,1,Cavalry FC,L,0,93,1
1,I91,Cavalry FC,W,2,Forge FC,L,1,91,1
2,I63,Forge FC,W,1,Cavalry FC,L,0,63,1
3,I28,Cavalry FC,L,0,Forge FC,W,1,28,1
4,I10,Forge FC,L,1,Cavalry FC,W,2,10,1


In [13]:
c=[]
for i in range(db.shape[0]):
    q1 = db.loc[i]['home']
    q2 = db.loc[i]['away']
    t1_x,t1_y = cpl.get_NB_data(db.head(i),q1)
    t2_x,t2_y = cpl.get_NB_data(db.head(i),q2)
    home_win, draw, away_win = cpl.get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)
    print(f'{q1} home win probability: {round(home_win,3)} \n{q2} away win probability: {round(away_win,3)} \nDraw probability: {round(draw,3)}')
    if home_win > away_win:
        hp, ap = 3, 1
    elif home_win < away_win:
        hp, ap = 1, 3
    else:
        hp, ap = 2, 2
    #print(hp,ap)
    c.append(hp)
    c.append(ap)

Forge FC home win probability: 0.33 
York9 FC away win probability: 0.33 
Draw probability: 0.33
Pacific FC home win probability: 0.33 
HFX Wanderers FC away win probability: 0.33 
Draw probability: 0.33
Pacific FC home win probability: 0.33 
Valour FC away win probability: 0.33 
Draw probability: 0.33
HFX Wanderers FC home win probability: 0.08 
Forge FC away win probability: 0.18 
Draw probability: 0.08
Cavalry FC home win probability: 0.33 
York9 FC away win probability: 0.33 
Draw probability: 0.33
Valour FC home win probability: 0.33 
FC Edmonton away win probability: 0.33 
Draw probability: 0.33
Forge FC home win probability: 0.13 
Pacific FC away win probability: 0.31 
Draw probability: 0.3
Cavalry FC home win probability: 0.68 
Valour FC away win probability: 0.31 
Draw probability: 0.18
Valour FC home win probability: 0.21 
HFX Wanderers FC away win probability: 0.31 
Draw probability: 0.23
Forge FC home win probability: 0.32 
Cavalry FC away win probability: 0.7 
Draw probabi

York9 FC home win probability: 0.37 
Cavalry FC away win probability: 0.62 
Draw probability: 0.34
Cavalry FC home win probability: 0.62 
Valour FC away win probability: 0.3 
Draw probability: 0.38
FC Edmonton home win probability: 0.38 
Forge FC away win probability: 0.62 
Draw probability: 0.36
Pacific FC home win probability: 0.31 
HFX Wanderers FC away win probability: 0.31 
Draw probability: 0.3
Valour FC home win probability: 0.26 
FC Edmonton away win probability: 0.42 
Draw probability: 0.28
Cavalry FC home win probability: 0.62 
HFX Wanderers FC away win probability: 0.35 
Draw probability: 0.38
Forge FC home win probability: 0.58 
York9 FC away win probability: 0.42 
Draw probability: 0.38
HFX Wanderers FC home win probability: 0.29 
Pacific FC away win probability: 0.36 
Draw probability: 0.32
Cavalry FC home win probability: 0.63 
Forge FC away win probability: 0.62 
Draw probability: 0.38
York9 FC home win probability: 0.35 
Forge FC away win probability: 0.59 
Draw probab

New England home win probability: 0.24 
Atlanta away win probability: 0.13 
Draw probability: 0.27
FC Dallas home win probability: 0.37 
Portland away win probability: 0.18 
Draw probability: 0.3
Colorado home win probability: 0.09 
D.C. United away win probability: 0.62 
Draw probability: 0.21
Real Salt Lake home win probability: 0.23 
Orlando City away win probability: 0.51 
Draw probability: 0.26
LA Galaxy home win probability: 0.55 
Philadelphia away win probability: 0.52 
Draw probability: 0.3
Los Angeles FC home win probability: 0.76 
FC Cincinnati away win probability: 0.47 
Draw probability: 0.3
Sporting KC home win probability: 0.63 
NY Red Bulls away win probability: 0.37 
Draw probability: 0.43
Vancouver home win probability: 0.18 
Los Angeles FC away win probability: 0.76 
Draw probability: 0.29
FC Cincinnati home win probability: 0.34 
Real Salt Lake away win probability: 0.34 
Draw probability: 0.33
Toronto FC home win probability: 0.46 
Minnesota away win probability: 0.

Los Angeles FC home win probability: 0.7 
Montreal away win probability: 0.46 
Draw probability: 0.41
Chicago home win probability: 0.38 
NYCFC away win probability: 0.6 
Draw probability: 0.35
Vancouver home win probability: 0.36 
FC Dallas away win probability: 0.32 
Draw probability: 0.36
New England home win probability: 0.3 
D.C. United away win probability: 0.53 
Draw probability: 0.3
FC Cincinnati home win probability: 0.22 
NY Red Bulls away win probability: 0.48 
Draw probability: 0.26
Philadelphia home win probability: 0.52 
Portland away win probability: 0.34 
Draw probability: 0.38
Minnesota home win probability: 0.35 
Houston away win probability: 0.5 
Draw probability: 0.32
Colorado home win probability: 0.08 
Columbus away win probability: 0.24 
Draw probability: 0.2
Sporting KC home win probability: 0.36 
Seattle away win probability: 0.72 
Draw probability: 0.38
Toronto FC home win probability: 0.32 
San Jose away win probability: 0.42 
Draw probability: 0.29
Atlanta h

Seattle home win probability: 0.49 
Atlanta away win probability: 0.36 
Draw probability: 0.37
NY Red Bulls home win probability: 0.43 
NYCFC away win probability: 0.66 
Draw probability: 0.34
New England home win probability: 0.4 
Vancouver away win probability: 0.41 
Draw probability: 0.36
Atlanta home win probability: 0.34 
Houston away win probability: 0.32 
Draw probability: 0.29
Chicago home win probability: 0.34 
Columbus away win probability: 0.16 
Draw probability: 0.34
Toronto FC home win probability: 0.36 
NY Red Bulls away win probability: 0.5 
Draw probability: 0.33
FC Cincinnati home win probability: 0.2 
D.C. United away win probability: 0.52 
Draw probability: 0.24
Portland home win probability: 0.34 
Orlando City away win probability: 0.42 
Draw probability: 0.3
LA Galaxy home win probability: 0.39 
Los Angeles FC away win probability: 0.7 
Draw probability: 0.3
Columbus home win probability: 0.2 
Montreal away win probability: 0.38 
Draw probability: 0.24
Toronto FC h

...

FC Dallas home win probability: 0.44 
Seattle away win probability: 0.48 
Draw probability: 0.36
Houston home win probability: 0.35 
LA Galaxy away win probability: 0.44 
Draw probability: 0.33
Seattle home win probability: 0.44 
Real Salt Lake away win probability: 0.4 
Draw probability: 0.35
Vancouver home win probability: 0.34 
Portland away win probability: 0.46 
Draw probability: 0.32
LA Galaxy home win probability: 0.39 
FC Dallas away win probability: 0.49 
Draw probability: 0.34
Colorado home win probability: 0.34 
Houston away win probability: 0.4 
Draw probability: 0.32
Sporting KC home win probability: 0.42 
San Jose away win probability: 0.36 
Draw probability: 0.36
NYCFC home win probability: 0.46 
Columbus away win probability: 0.41 
Draw probability: 0.36
Orlando City home win probability: 0.32 
D.C. United away win probability: 0.47 
Draw probability: 0.32
New England home win probability: 0.38 
Montreal away win probability: 0.4 
Draw probability: 0.34
Toronto FC home 

Orlando City home win probability: 0.33 
Toronto FC away win probability: 0.49 
Draw probability: 0.32
Seattle home win probability: 0.44 
Portland away win probability: 0.46 
Draw probability: 0.35
Houston home win probability: 0.36 
FC Dallas away win probability: 0.5 
Draw probability: 0.34
Real Salt Lake home win probability: 0.38 
San Jose away win probability: 0.35 
Draw probability: 0.32
Philadelphia home win probability: 0.38 
Toronto FC away win probability: 0.49 
Draw probability: 0.34
D.C. United home win probability: 0.41 
Columbus away win probability: 0.42 
Draw probability: 0.36
New England home win probability: 0.39 
NY Red Bulls away win probability: 0.5 
Draw probability: 0.34
Portland home win probability: 0.41 
Vancouver away win probability: 0.38 
Draw probability: 0.34
LA Galaxy home win probability: 0.4 
Colorado away win probability: 0.4 
Draw probability: 0.34
Sporting KC home win probability: 0.42 
Chicago away win probability: 0.38 
Draw probability: 0.36
NYC

Orlando City home win probability: 0.33 
Colorado away win probability: 0.4 
Draw probability: 0.32
Columbus home win probability: 0.37 
New England away win probability: 0.44 
Draw probability: 0.34
NY Red Bulls home win probability: 0.44 
Real Salt Lake away win probability: 0.4 
Draw probability: 0.35
Toronto FC home win probability: 0.44 
Montreal away win probability: 0.38 
Draw probability: 0.36
Chicago home win probability: 0.34 
D.C. United away win probability: 0.47 
Draw probability: 0.32
LA Galaxy home win probability: 0.4 
Portland away win probability: 0.46 
Draw probability: 0.35
FC Dallas home win probability: 0.45 
Houston away win probability: 0.4 
Draw probability: 0.36
Toronto FC home win probability: 0.45 
D.C. United away win probability: 0.47 
Draw probability: 0.36
Philadelphia home win probability: 0.38 
Montreal away win probability: 0.38 
Draw probability: 0.33
New England home win probability: 0.38 
Vancouver away win probability: 0.38 
Draw probability: 0.34

Portland home win probability: 0.42 
Houston away win probability: 0.4 
Draw probability: 0.34
Toronto FC home win probability: 0.44 
Orlando City away win probability: 0.36 
Draw probability: 0.36
D.C. United home win probability: 0.41 
San Jose away win probability: 0.36 
Draw probability: 0.36
Columbus home win probability: 0.38 
Sporting KC away win probability: 0.48 
Draw probability: 0.34
Montreal home win probability: 0.34 
Philadelphia away win probability: 0.42 
Draw probability: 0.31
Chicago home win probability: 0.34 
Colorado away win probability: 0.4 
Draw probability: 0.32
Vancouver home win probability: 0.36 
FC Dallas away win probability: 0.5 
Draw probability: 0.32
Real Salt Lake home win probability: 0.36 
Seattle away win probability: 0.46 
Draw probability: 0.32
LA Galaxy home win probability: 0.4 
NYCFC away win probability: 0.49 
Draw probability: 0.35
Chicago home win probability: 0.34 
NY Red Bulls away win probability: 0.5 
Draw probability: 0.32
Colorado home

Montreal home win probability: 0.36 
Toronto FC away win probability: 0.49 
Draw probability: 0.32
NYCFC home win probability: 0.44 
New England away win probability: 0.44 
Draw probability: 0.36
Philadelphia home win probability: 0.38 
Orlando City away win probability: 0.36 
Draw probability: 0.33
Columbus home win probability: 0.38 
D.C. United away win probability: 0.46 
Draw probability: 0.34
Portland home win probability: 0.42 
Colorado away win probability: 0.38 
Draw probability: 0.34
Vancouver home win probability: 0.36 
Houston away win probability: 0.4 
Draw probability: 0.32
Seattle home win probability: 0.43 
Real Salt Lake away win probability: 0.39 
Draw probability: 0.34
Sporting KC home win probability: 0.42 
LA Galaxy away win probability: 0.46 
Draw probability: 0.36
FC Dallas home win probability: 0.45 
San Jose away win probability: 0.36 
Draw probability: 0.36
Chicago home win probability: 0.32 
NY Red Bulls away win probability: 0.5 
Draw probability: 0.32


In [14]:
print(len(c),results.shape)

3866 (3866, 18)


In [23]:
d=[]
for i in range(db.shape[0]):
    q1 = db.loc[i]['home']
    q2 = db.loc[i]['away']
    games = get_team_comparison(db.head(i),q1,q2)
    t1_x,t1_y = cpl.get_NB_data(games,q1)
    t2_x,t2_y = cpl.get_NB_data(games,q2)
    home_win, draw, away_win = cpl.get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)
    print(f'{q1} home win probability: {round(home_win,3)} \n{q2} away win probability: {round(away_win,3)} \nDraw probability: {round(draw,3)}')
    if home_win > away_win:
        hp, ap = 3, 1
    elif home_win < away_win:
        hp, ap = 1, 3
    else:
        hp, ap = 2, 2
    #print(hp,ap)
    d.append(hp)
    d.append(ap)

Forge FC home win probability: 0.18 
York9 FC away win probability: 0.92 
Draw probability: 0.68
Pacific FC home win probability: 0.18 
HFX Wanderers FC away win probability: 0.92 
Draw probability: 0.68
Pacific FC home win probability: 0.18 
Valour FC away win probability: 0.92 
Draw probability: 0.68
HFX Wanderers FC home win probability: 0.18 
Forge FC away win probability: 0.92 
Draw probability: 0.68
Cavalry FC home win probability: 0.18 
York9 FC away win probability: 0.92 
Draw probability: 0.68
Valour FC home win probability: 0.18 
FC Edmonton away win probability: 0.92 
Draw probability: 0.68
Forge FC home win probability: 0.18 
Pacific FC away win probability: 0.92 
Draw probability: 0.68
Cavalry FC home win probability: 0.18 
Valour FC away win probability: 0.92 
Draw probability: 0.68
Valour FC home win probability: 0.18 
HFX Wanderers FC away win probability: 0.92 
Draw probability: 0.68
Forge FC home win probability: 0.18 
Cavalry FC away win probability: 0.92 
Draw proba

York9 FC home win probability: 0.04 
Cavalry FC away win probability: 0.72 
Draw probability: 0.04
Cavalry FC home win probability: 0.8 
Valour FC away win probability: 0.1 
Draw probability: 0.4
FC Edmonton home win probability: 0.4 
Forge FC away win probability: 0.44 
Draw probability: 0.32
Pacific FC home win probability: 0.43 
HFX Wanderers FC away win probability: 0.21 
Draw probability: 0.29
Valour FC home win probability: 0.55 
FC Edmonton away win probability: 0.24 
Draw probability: 0.38
Cavalry FC home win probability: 0.77 
HFX Wanderers FC away win probability: 0.08 
Draw probability: 0.34
Forge FC home win probability: 0.58 
York9 FC away win probability: 0.35 
Draw probability: 0.33
HFX Wanderers FC home win probability: 0.33 
Pacific FC away win probability: 0.4 
Draw probability: 0.3
Cavalry FC home win probability: 0.21 
Forge FC away win probability: 0.43 
Draw probability: 0.23
York9 FC home win probability: 0.2 
Forge FC away win probability: 0.64 
Draw probability

Houston home win probability: 0.18 
San Jose away win probability: 0.92 
Draw probability: 0.68
Seattle home win probability: 0.18 
Toronto FC away win probability: 0.92 
Draw probability: 0.68
Minnesota home win probability: 0.18 
NYCFC away win probability: 0.92 
Draw probability: 0.68
New England home win probability: 0.18 
Atlanta away win probability: 0.92 
Draw probability: 0.68
FC Dallas home win probability: 0.18 
Portland away win probability: 0.92 
Draw probability: 0.68
Colorado home win probability: 0.18 
D.C. United away win probability: 0.92 
Draw probability: 0.68
Real Salt Lake home win probability: 0.18 
Orlando City away win probability: 0.92 
Draw probability: 0.68
LA Galaxy home win probability: 0.18 
Philadelphia away win probability: 0.92 
Draw probability: 0.68
Los Angeles FC home win probability: 0.18 
FC Cincinnati away win probability: 0.92 
Draw probability: 0.68
Sporting KC home win probability: 0.18 
NY Red Bulls away win probability: 0.92 
Draw probability

NY Red Bulls home win probability: 0.18 
Atlanta away win probability: 0.92 
Draw probability: 0.68
FC Dallas home win probability: 0.08 
Los Angeles FC away win probability: 0.68 
Draw probability: 0.08
LA Galaxy home win probability: 0.18 
Colorado away win probability: 0.92 
Draw probability: 0.68
NY Red Bulls home win probability: 0.18 
Vancouver away win probability: 0.92 
Draw probability: 0.68
Orlando City home win probability: 0.18 
LA Galaxy away win probability: 0.92 
Draw probability: 0.68
Real Salt Lake home win probability: 0.18 
Atlanta away win probability: 0.92 
Draw probability: 0.68
Los Angeles FC home win probability: 0.18 
Montreal away win probability: 0.92 
Draw probability: 0.68
Chicago home win probability: 0.08 
NYCFC away win probability: 0.68 
Draw probability: 0.08
Vancouver home win probability: 0.18 
FC Dallas away win probability: 0.92 
Draw probability: 0.68
New England home win probability: 0.18 
D.C. United away win probability: 0.92 
Draw probability:

Montreal home win probability: 0.18 
Toronto FC away win probability: 0.92 
Draw probability: 0.68
Chicago home win probability: 0.18 
FC Cincinnati away win probability: 0.92 
Draw probability: 0.68
Minnesota home win probability: 0.18 
FC Dallas away win probability: 0.92 
Draw probability: 0.68
Vancouver home win probability: 0.92 
Sporting KC away win probability: 0.18 
Draw probability: 0.92
Real Salt Lake home win probability: 0.18 
Philadelphia away win probability: 0.92 
Draw probability: 0.68
Portland home win probability: 0.92 
Colorado away win probability: 0.18 
Draw probability: 0.92
Seattle home win probability: 0.18 
Atlanta away win probability: 0.92 
Draw probability: 0.68
NY Red Bulls home win probability: 0.18 
NYCFC away win probability: 0.92 
Draw probability: 0.68
New England home win probability: 0.18 
Vancouver away win probability: 0.92 
Draw probability: 0.68
Atlanta home win probability: 0.18 
Houston away win probability: 0.92 
Draw probability: 0.68
Chicago

...

NYCFC home win probability: 0.38 
D.C. United away win probability: 0.54 
Draw probability: 0.33
Vancouver home win probability: 0.38 
NY Red Bulls away win probability: 0.76 
Draw probability: 0.42
New England home win probability: 0.43 
Colorado away win probability: 0.21 
Draw probability: 0.29
Chicago home win probability: 0.23 
Philadelphia away win probability: 0.4 
Draw probability: 0.23
FC Dallas home win probability: 0.44 
Portland away win probability: 0.5 
Draw probability: 0.38
LA Galaxy home win probability: 0.21 
Columbus away win probability: 0.43 
Draw probability: 0.23
Montreal home win probability: 0.5 
Orlando City away win probability: 0.22 
Draw probability: 0.34
Real Salt Lake home win probability: 0.25 
LA Galaxy away win probability: 0.45 
Draw probability: 0.26
Sporting KC home win probability: 0.44 
Houston away win probability: 0.4 
Draw probability: 0.36
Columbus home win probability: 0.52 
Vancouver away win probability: 0.44 
Draw probability: 0.46
Philade

Sporting KC home win probability: 0.4 
NY Red Bulls away win probability: 0.44 
Draw probability: 0.32
Seattle home win probability: 0.37 
New England away win probability: 0.82 
Draw probability: 0.44
Houston home win probability: 0.84 
Orlando City away win probability: 0.13 
Draw probability: 0.51
Chicago home win probability: 0.24 
Vancouver away win probability: 0.62 
Draw probability: 0.28
Columbus home win probability: 0.31 
Toronto FC away win probability: 0.66 
Draw probability: 0.34
FC Dallas home win probability: 0.5 
Sporting KC away win probability: 0.3 
Draw probability: 0.36
Real Salt Lake home win probability: 0.5 
Philadelphia away win probability: 0.15 
Draw probability: 0.3
Seattle home win probability: 0.75 
San Jose away win probability: 0.19 
Draw probability: 0.41
NYCFC home win probability: 0.34 
New England away win probability: 0.54 
Draw probability: 0.33
Portland home win probability: 0.38 
LA Galaxy away win probability: 0.42 
Draw probability: 0.33
New Eng

Toronto FC home win probability: 0.36 
Houston away win probability: 0.4 
Draw probability: 0.34
NY Red Bulls home win probability: 0.33 
NYCFC away win probability: 0.43 
Draw probability: 0.3
D.C. United home win probability: 0.46 
Orlando City away win probability: 0.24 
Draw probability: 0.33
NYCFC home win probability: 0.55 
Chicago away win probability: 0.3 
Draw probability: 0.4
FC Dallas home win probability: 0.08 
NY Red Bulls away win probability: 0.78 
Draw probability: 0.16
Montreal home win probability: 0.84 
Real Salt Lake away win probability: 0.13 
Draw probability: 0.51
Vancouver home win probability: 0.18 
Seattle away win probability: 0.65 
Draw probability: 0.26
New England home win probability: 0.28 
Toronto FC away win probability: 0.55 
Draw probability: 0.29
Houston home win probability: 0.24 
Portland away win probability: 0.63 
Draw probability: 0.28
San Jose home win probability: 0.24 
Columbus away win probability: 0.62 
Draw probability: 0.28
Orlando City h

NY Red Bulls home win probability: 0.32 
New England away win probability: 0.42 
Draw probability: 0.28
Philadelphia home win probability: 0.04 
Portland away win probability: 0.71 
Draw probability: 0.04
Orlando City home win probability: 0.24 
FC Dallas away win probability: 0.62 
Draw probability: 0.28
Montreal home win probability: 0.42 
Columbus away win probability: 0.32 
Draw probability: 0.34
Chicago home win probability: 0.36 
Seattle away win probability: 0.4 
Draw probability: 0.34
Colorado home win probability: 0.24 
Real Salt Lake away win probability: 0.52 
Draw probability: 0.27
NYCFC home win probability: 0.42 
Toronto FC away win probability: 0.6 
Draw probability: 0.38
Vancouver home win probability: 0.26 
Sporting KC away win probability: 0.55 
Draw probability: 0.29
Chicago home win probability: 0.44 
Columbus away win probability: 0.66 
Draw probability: 0.43
LA Galaxy home win probability: 0.35 
San Jose away win probability: 0.47 
Draw probability: 0.33
Toronto F

Columbus home win probability: 0.8 
FC Dallas away win probability: 0.1 
Draw probability: 0.4
Vancouver home win probability: 0.36 
Colorado away win probability: 0.56 
Draw probability: 0.34
Portland home win probability: 0.46 
Sporting KC away win probability: 0.48 
Draw probability: 0.41
NY Red Bulls home win probability: 0.42 
Chicago away win probability: 0.28 
Draw probability: 0.32
FC Dallas home win probability: 0.37 
NYCFC away win probability: 0.82 
Draw probability: 0.44
Houston home win probability: 0.3 
Real Salt Lake away win probability: 0.5 
Draw probability: 0.3
Colorado home win probability: 0.21 
D.C. United away win probability: 0.64 
Draw probability: 0.29
LA Galaxy home win probability: 0.5 
Montreal away win probability: 0.15 
Draw probability: 0.3
San Jose home win probability: 0.28 
Seattle away win probability: 0.62 
Draw probability: 0.3
Toronto FC home win probability: 0.46 
New England away win probability: 0.32 
Draw probability: 0.36
Orlando City home wi

In [24]:
print(len(d),results.shape)

3866 (3866, 19)


In [25]:
results['pred'] = d

In [26]:
s,t = [],[]
for i in range(results.shape[0]):
    if results.loc[i]['pred'] == results.loc[i]['r']:
        t.append(1)
    else:
        s.append(1)
print(round(len(s)/results.shape[0],2),round(len(t)/results.shape[0],2))

0.76 0.24


In [26]:
results.to_csv('datasets/soccer-nn-train-pred.csv',index=False)

In [27]:
results.head(50)

,game,team,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,r,s,pred
0,I1,Forge FC,0.71,0.89,0.74,0.71,0.48,0.89,0.77,0.52,0.50,0.48,0.42,0.0,0.0,0.0,2,1,2
1,I1,York9 FC,0.93,0.92,0.91,0.86,0.41,0.78,0.65,0.64,0.46,0.70,0.47,0.0,0.0,0.0,2,1,2
2,I2,Pacific FC,0.54,0.78,0.69,0.50,0.39,0.72,0.69,0.57,0.51,0.79,0.27,0.0,0.0,0.0,3,1,2
3,I2,HFX Wanderers FC,0.54,0.92,0.77,0.67,0.62,0.57,0.52,0.47,0.37,0.77,0.28,0.0,0.0,0.0,1,0,2
4,I3,Pacific FC,0.54,0.78,0.69,0.50,0.39,0.72,0.69,0.57,0.51,0.79,0.27,0.0,0.0,0.0,1,1,2
5,I3,Valour FC,0.55,0.73,0.71,0.60,0.45,0.90,0.64,0.60,0.41,0.30,0.25,0.0,0.0,0.0,3,2,2
6,I4,HFX Wanderers FC,0.54,0.92,0.77,0.67,0.62,0.57,0.52,0.47,0.37,0.77,0.28,0.0,0.0,0.0,3,2,1
7,I4,Forge FC,0.71,0.89,0.74,0.71,0.48,0.89,0.77,0.52,0.50,0.48,0.42,0.0,0.0,0.0,1,1,3
8,I5,Cavalry FC,0.79,0.91,0.75,0.37,0.33,0.87,0.66,0.66,0.62,0.73,0.64,0.0,0.0,0.0,3,2,2
9,I5,York9 FC,0.93,0.92,0.91,0.86,0.41,0.78,0.65,0.64,0.46,0.70,0.47,0.0,0.0,0.0,1,1,2
